In [63]:
import pandas as pd
import re

In [64]:
df = pd.read_csv('fundacurrentlistings20240517_dataset.csv')

In [65]:
df = df.rename(columns={'num_of_bathrooms':'num_of_floors'})
df.describe()

,url,price,address,descrip,zip_code,size,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking,last_ask_price,city
count,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951,951
unique,951,250,922,844,593,161,170,174,59,3,31,29,605,51,56,40,43,18,357,1
top,https://www.funda.nl/huur/amsterdam/appartemen...,€ 2.750 /mnd,na,na,1043 BV Amsterdam,na,2024,na,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na,na,amsterdam
freq,1,36,20,20,153,136,232,67,256,623,375,295,101,374,496,540,618,849,65,951


In [66]:
df = df.drop_duplicates()
df = df.drop(columns={'size', 'city', 'last_ask_price'})

# clean the price
df['price'] = df['price'].str.replace('€ ', '')
df['price'] = df['price'].str.replace(' /mnd', '')
df['price'] = df['price'].str.replace('k', '')
df['price'] = df['price'].str.replace('na', '0')
df['price'] = df['price'].str.replace('Huurprijs op aanvraag', '0')
df['price'] = df['price'].str.replace('.', '')

# drop all rows where dependent variable price is equal to zero and higher than 12500 euros a months. Those are buy listings listed wrongly
df = df.drop(df[df['price'] == '0'].index)
df = df.drop(df[pd.to_numeric(df['price']) > 12500].index)
# df.sort_values('price', ascending=False, key=pd.to_numeric)
df.describe()

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking
count,921,921,921,921,921,921,921,921,921,921,921,921,921,921,921,921,921
unique,921,239,912,833,582,169,171,58,3,27,26,597,51,56,40,41,16
top,https://www.funda.nl/huur/amsterdam/appartemen...,2750,Reguliersgracht 15 3,NARITAWEG - HET HART VAN VOORUITZIEND AMSTE...,1043 BV Amsterdam,2024,70 m²,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na
freq,1,36,2,6,153,232,60,255,615,373,295,79,347,469,513,591,822


In [67]:
# clean zip_code. Should be len = 6 with 4 numbers and 2 letters
df['zip_code'] = df['zip_code'].str[:7]
df['zip_code'] = df['zip_code'].str.replace(' ', '')

In [68]:
# remove non-houses
df = df.drop(df[df['kind_of_house'] == 'Garage'].index)
df = df.drop(df[df['kind_of_house'] == 'Inpandige garage'].index)
df = df.drop(df[df['kind_of_house'] == 'parkeerkelder'].index)
df = df.drop(df[df['kind_of_house'] == 'Parkeerplaats'].index)
df.describe()

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking
count,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893,893
unique,893,226,885,805,563,169,165,55,3,27,26,597,51,56,40,41,16
top,https://www.funda.nl/huur/amsterdam/appartemen...,2750,Reguliersgracht 15 3,NARITAWEG - HET HART VAN VOORUITZIEND AMSTE...,1043BV,2024,70 m²,Bovenwoning (appartement),Bestaande bouw,3 kamers (2 slaapkamers),1 woonlaag,na,na,na,na,na,na
freq,1,36,2,6,153,232,60,255,588,373,295,51,319,441,485,563,794


In [69]:
df['living_area'] = df['living_area'].str.replace(' m²', '')
df['living_area'] = df['living_area'].str.replace('na', '')

In [70]:
df.head()

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,num_of_floors,layout,energy_label,insulation,heating,ownership,parking
0,https://www.funda.nl/huur/amsterdam/appartemen...,1045,Schipluidenlaan 254,Podium * VERHUUR GESTART * 147 middenhuur ...,1062HE,2024,44,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),6 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,na,na
1,https://www.funda.nl/huur/amsterdam/appartemen...,1068,Krijn Taconiskade 299,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),6 woonlagen,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na
2,https://www.funda.nl/huur/amsterdam/appartemen...,1099,Haarlemmerweg,** BINNENKORT IN VERHUUR | WesterparkWest f...,1014BL,2024,50,Galerijflat,Nieuwbouw,2 kamers (1 slaapkamer),10 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,Nee,na
3,https://www.funda.nl/huur/amsterdam/appartemen...,1103,Krijn Taconiskade 441,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),6 woonlagen,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na
4,https://www.funda.nl/huur/amsterdam/appartemen...,1105,Willem Frogerstraat 41,Podium *VERHUUR GESTART* 147 middenhuur app...,1062HZ,2024,56,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),6 woonlagen,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na,na,na


In [77]:
# extract number of rooms in total
df['num_of_rooms'] = df['layout'].str.extract(r'Aantal kamers(\d+)')
# extract the number of bathrooms
df['num_of_bedrooms'] = df['layout'].str.extract(r'(\d+) slaapkamer')
# extract the number of bedrooms
df['num_of_bathrooms'] = df['layout'].str.extract(r'(\d+) badkamer')
# extract the number of seperate toilets
df['num_of_separate_toilets'] = df['layout'].str.extract(r'(\d+) apart')
# extract the number of floors
df['total_num_of_floors'] = df['layout'].str.extract(r'Aantal woonlagen(\d+)')
# correct the number of floors. It cannot be lower than the floor number of the house

# extract the floor number of house
df['floor_of_house'] = df['layout'].str.extract(r'Gelegen op(\w)')



In [78]:
df.head(50)

,url,price,address,descrip,zip_code,year,living_area,kind_of_house,building_type,num_of_rooms,...,energy_label,insulation,heating,ownership,parking,num_of_bedrooms,num_of_bathrooms,num_of_separate_toilets,total_num_of_floors,floor_of_house
0,https://www.funda.nl/huur/amsterdam/appartemen...,1045,Schipluidenlaan 254,Podium * VERHUUR GESTART * 147 middenhuur ...,1062HE,2024,44,Portiekflat,Nieuwbouw,2,...,na,na,na,na,na,1,NaN,NaN,6,1
1,https://www.funda.nl/huur/amsterdam/appartemen...,1068,Krijn Taconiskade 299,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1,...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na,1,NaN,NaN,6,3
2,https://www.funda.nl/huur/amsterdam/appartemen...,1099,Haarlemmerweg,** BINNENKORT IN VERHUUR | WesterparkWest f...,1014BL,2024,50,Galerijflat,Nieuwbouw,2,...,na,na,na,Nee,na,1,NaN,NaN,10,NaN
3,https://www.funda.nl/huur/amsterdam/appartemen...,1103,Krijn Taconiskade 441,*** See English version below *** Huren D...,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1,...,A++,Eco-bouw en volledig geïsoleerd,Gehele vloerverwarming,na,na,1,NaN,NaN,6,5
4,https://www.funda.nl/huur/amsterdam/appartemen...,1105,Willem Frogerstraat 41,Podium *VERHUUR GESTART* 147 middenhuur app...,1062HZ,2024,56,Portiekflat,Nieuwbouw,2,...,na,na,na,na,na,1,NaN,NaN,6,1
5,https://www.funda.nl/huur/amsterdam/appartemen...,1115,Schipluidenlaan 270,Podium * VERHUUR GESTART * 147 middenhuur ...,1062HE,2024,50,Portiekflat,Nieuwbouw,2,...,na,na,na,na,na,1,NaN,NaN,6,1
6,https://www.funda.nl/huur/amsterdam/appartemen...,1120,Staalmeesterslaan 377,"Via de website van Makelaardij Hoekstra, hu...",1057PG,1971-1980,58,Galerijflat (appartement),Bestaande bouw,2,...,A+,na,na,na,na,1,NaN,NaN,15,1
7,https://www.funda.nl/huur/amsterdam/appartemen...,1120,Staalmeesterslaan 380,"Via de website van Makelaardij Hoekstra, hu...",1057PG,1971-1980,58,Galerijflat (appartement),Bestaande bouw,2,...,A+,na,na,na,na,1,NaN,NaN,15,1
8,https://www.funda.nl/huur/amsterdam/appartemen...,1130,Olga de Haasstraat 513,Living the high life! *VERHUUR GESTART* Sch...,1095PG,2024,58,Portiekflat,Nieuwbouw,2,...,na,na,na,na,na,1,NaN,NaN,32,2
9,https://www.funda.nl/huur/amsterdam/appartemen...,1135,Willem Frogerstraat 53,Podium * VERHUUR GESTART * 147 middenhuur ...,1062HZ,2024,80,Portiekflat,Nieuwbouw,4,...,na,na,na,na,na,3,NaN,NaN,1,1
